In [24]:
from pathlib import Path
import sys
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
# Path to project root
PROJECT_ROOT = Path.home() / "Desktop" / "Insurance_Project"

# Add root to Python path
sys.path.append(str(PROJECT_ROOT))

In [25]:
from pathlib import Path
import os
import importlib
from src.data import preprocessing as pr
file_path = PROJECT_ROOT/"data"/"raw"/"insurance_claims.csv"
data = pr.load_data(file_path)
data = data.drop('_c39', axis=1)

In [26]:
null_count = int(data.isna().sum().sum())
print(f"Null count: {null_count}")

Null count: 91


In [28]:
importlib.reload(pr)
clean_data = pr.clean_data(data)

In [35]:
clean_data['authorities_contacted'].isna().sum()

np.int64(0)

In [37]:
null_count_cleaned = int(clean_data.isna().sum().sum())
print(f"Null count: {null_count_cleaned}")

Null count: 0
